# 유튜브 크롤링
제목, 링크, 댓글 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

path = './chromedriver'

In [2]:
driver = webdriver.Chrome(path)
keyword = '이이제이'
url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
driver.get(url)
driver.maximize_window()

In [3]:
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

In [4]:
# 필터 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
time.sleep(1)
# 구분 -> 동영상 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

In [12]:
# 제목 & 링크 크롤링
for i in range(10):
    scroll_down(driver)
    
time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')
name = soup.select('a#video-title')
video_url = soup.select('a#video-title')

video_num_list = []
name_list = []
url_list = []

for i in range(len(name)):
    video_num_list.append(i+1)
    name_list.append(name[i].text.strip())
for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))
    
    
youtubeDic = {
    '영상 번호' : video_num_list,
    '제목': name_list,
    '주소': url_list,
}

youtubeDf = pd.DataFrame(youtubeDic)

youtubeDf.head()

,영상 번호,제목,주소
0,1,[이이제이x라이브!] 국민들은 콩가루 집안을 싫어합니다,https://www.youtube.com/watch?v=idF9WIhqJbs
1,2,"[이이제이x라이브!] 권순정 팀장님, 여론조사 말고 이재명 캠프이야기나 해줘요!",https://www.youtube.com/watch?v=RRiQoWDrpRw
2,3,[저널리즘M2] 제아무리 당대표라도 윤핵관을 지날 수는 없소!,https://www.youtube.com/watch?v=bklsX8K4LQg
3,4,[이이제이x라이브!] #이재명은합니다! #우리도합니다!,https://www.youtube.com/watch?v=DlXnKM4pwNk
4,5,[이이제이x라이브!] 현재 지지율? 아무런 의미 없다!,https://www.youtube.com/watch?v=S0-vuTdTVAM


In [14]:
# 댓글 크롤링
video_num = 0
video_idex=[]
video_comm = []


for i in range(2):
    driver.get(url_list[i])
    
    time.sleep(5)
    
    try :
    # 동영상 재생 멈추기 - 광고로 시작하면 적용 안되는 문제
        driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[1]/div/div/div/ytd-player/div/div/div[1]/video').click()
    except:
        pass
    try: # 광고로 시작할 때 동영상 재생 멈추기
        driver.find_element_by_xpath('').click()
    except:
        pass
        
    # 유튜브 프리미엄 팝업창 닫기
    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass
    
    # 댓글 보이게 하기(밑으로 내리기)
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,800);")
    time.sleep(3)
    
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.5)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(1.5)
    
    
    # 댓글 더보기 클릭 (대댓글)
    buttons = driver.find_elements_by_css_selector("#more-replies > a")
    time.sleep(1.5)
    for button in buttons:
        button.send_keys(Keys.ENTER)
        time.sleep(1.5)
        button.click()
        
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    comment_list = soup.select("yt-formatted-string#content-text")

    comment_final = []

    for i in range(len(comment_list)):
        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        temp_comment = temp_comment.replace('\xa0@', '')
        temp_comment = temp_comment.replace('\xa0', '')
        temp_comment = temp_comment.replace('\r', '')
        comment_final.append(temp_comment)
    
    video_idex.append(video_num)
    video_comm.append(comment_final)
    video_num += 1

In [15]:
df = []
for i in range(len(video_idex)):
    if len(video_comm[i]) != 0:
        n = np.full((len(video_comm[i])),i+1)
        temp = pd.DataFrame([ x for x in zip(n,video_comm[i])])
        df.append(temp)

In [16]:
comment_df = df[0]
for i in range(1, len(df)):
    comment_df = pd.concat([comment_df, df[i]])
    
comment_df.columns = ['영상 번호', '댓글']
comment_df.head()

,영상 번호,댓글
0,1,바로 방송 시청하기는 4:41 입니다.구독과 좋아요는 언제나 큰 힘이 됩니다:) 감...
1,1,폭력과 분열 분쟁은 이제 그만! 동서통합 남북통일시대로 나아가자!부국강병 나라가 부...
2,1,이동형 작가님 보세요.성질더러운 이작가님의 조언대로기본소득ㆍ국토보유세ㆍ탄소세한발짝씩...
3,1,이이제이 재미있어서 매주 봅니다.
4,1,우리는 이동형 패밀리들이 있기때문에 희망이 있다~가즈아~~~~~~~!!


In [18]:
#CSV형태로 저장하기
youtubeDf.to_csv('./'+'유튜브_'+keyword+'_크롤링.csv', encoding='utf-8-sig', index=False)
comment_df.to_csv('./'+'유튜브_'+keyword+'_댓글.csv', encoding='utf-8-sig', index=False)